In [ ]:
#Update your token
STUDENT_TOKEN = 'GABRIEL DE OLAGUIBEL'

## ignore this code, just used for submission
import requests
import pprint
import json
import random
import math
from copy import copy, deepcopy

def get_problem(problem_id):
  r = requests.get('https://emarchiori.eu.pythonanywhere.com/get-problem?TOKEN=%s&problem=%s' % (STUDENT_TOKEN, problem_id))
  if not r.status_code == 200:
    print('\033[91m' + str(r.status_code))
  return r.json()

def submit_answer(problem_id, answer):
  r = requests.get('https://emarchiori.eu.pythonanywhere.com/submit-answer?TOKEN=%s&problem=%s' % (STUDENT_TOKEN, problem_id), json = answer)
  if not r.status_code == 200:
    print('\033[91m' + str(r.status_code))
  result = r.json()['result']
  if result == 'PASSED':
    print('\033[92m' + result)
  else:
    print('\033[91m' + result)

In [ ]:
def format_value(value):
  if value == '.':
    return '..'
  else:
    s = str(value)
    if len(s) == 1:
      s = ' ' + s
    return s

def format_h(const, max):
  s = ''
  for i in range(max - len(const)):
    s = s + '   '
  s = s + ' '.join(map(format_value, const))
  return s

def pretty_print(board, h_const, v_const):
  max_h = 1
  for c in h_const:
    max_h = max(max_h, len(c))
  max_v = 1
  for c in v_const:
    max_v = max(max_v, len(c))

  h_offset = ' '.join(map(lambda x: '  ', range(max_h)))
  for i in range(max_v):
    print('    ' + h_offset + ' '.join(map(lambda x: '  ' if i >= len(x) else format_value(x[i]), v_const)))
  for r in range(len(board)):
    row = board[r]
    print((h_offset if r ==0 else format_h(h_const[r-1], max_h)) + ' ' +  ' '.join(map(format_value, row)))

In [2]:
import itertools # using this library to generate all possible combinations of a list which is useful when applying constraints


# function to Determine the possible values for each empty cell in a sequence by considering all other cells.
def generate_possible_values(empty_indices, filled_cells, target_sum): 

    remaining_sum = target_sum - sum(filled_cells)
    
    # Exclud numbers already used in the filled cells when generating combinations
    available_numbers = [num for num in range(1, 10) if num not in filled_cells]
    
    # Generate all possible combinations of numbers that can fill the empty cells and match the remaining sum.
    valid_permutations = [seq for seq in itertools.permutations(available_numbers, len(empty_indices)) 
                          if sum(seq) == remaining_sum and all(num not in filled_cells for num in seq)] # checking if the numbers are not in the filled cells
    
    # For each cell position, determine the possible values it can take
    possible_values_per_cell = [set() for _ in empty_indices] 
    for perm in valid_permutations:
        for i, val in enumerate(perm): # i is index and val is obviously the value btw
            possible_values_per_cell[i].add(val)
    
    return possible_values_per_cell 

# function provides a sequence (horizontal or vertical) that the cell belongs to and its sum constraint
def find_sequence_and_sum(board, row, col, constraints, is_horizontal):
    if is_horizontal:
        start, end = col, col 
        while start > 0 and board[row][start - 1] != 'X': # checking if the cell were looking at is not the first one in the sequence
            start -= 1 # move left to find it
        while end < len(board[row]) and board[row][end] != 'X': # checking if the cell were looking at is not the last one in the sequence
            end += 1 # move right to find it
        sequence = board[row][start:end] 
        target_sum = constraints[row - 1][0] if row > 0 else None # if the row is 0 then there is no constraint
    else: # same thing but for vertical:
        start, end = row, row
        while start > 0 and board[start - 1][col] != 'X':
            start -= 1
        while end < len(board) and board[end][col] != 'X':
            end += 1
        sequence = [board[i][col] for i in range(start, end)]
        target_sum = constraints[col - 1][0] if col > 0 else None # if the col is 0 then there is no constraint

    return sequence, target_sum

# function updates the board with the possible values for each sequence given the filled cells and the sum constraint
def propagate_constraints(board, row, col, h_constraints, v_constraints): 
    # Find the sequemces affected by the placement of a number 
    h_word, h_target_sum = find_sequence_and_sum(board, row, col, h_constraints, True)
    v_word, v_target_sum = find_sequence_and_sum(board, row, col, v_constraints, False)

    # Initialize possibilities for the horizontal and vertical sequences
    # if the cell is empty then it can be any number from 1 to 9 initially
    h_possibilities = [set(range(1, 10)) if x == '.' else {int(x)} for x in h_word]
    v_possibilities = [set(range(1, 10)) if x == '.' else {int(x)} for x in v_word]

    # use generate_possible_values for the empty cells in the horizontal and vertical sequences
    if h_target_sum: # if there is a constraint
        h_empty_indices = [i for i, x in enumerate(h_word) if x == '.'] # geting the index of the empty cells
        h_filled_cells = [int(x) for x in h_word if x != '.'] # getting the filled cells
        h_possibilities_for_empty = generate_possible_values(h_empty_indices, h_filled_cells, h_target_sum) # calling the function
        for i, index in enumerate(h_empty_indices):
            h_possibilities[index] = h_possibilities_for_empty[i]

    if v_target_sum:
        v_empty_indices = [i for i, x in enumerate(v_word) if x == '.']
        v_filled_cells = [int(x) for x in v_word if x != '.']
        v_possibilities_for_empty = generate_possible_values(v_empty_indices, v_filled_cells, v_target_sum)
        for i, index in enumerate(v_empty_indices):
            v_possibilities[index] = v_possibilities_for_empty[i]

    # Get the index of the current cell in the horizontal and vertical sequences
    h_index = h_word.index('.') if '.' in h_word else None
    v_index = v_word.index('.') if '.' in v_word else None

    # Combine the possibilities for the current cell from the horizontal and vertical sequences
    combined_possibilities = set(range(1, 10)) # intital possibilities
    if h_index is not None and h_index < len(h_possibilities): 
        combined_possibilities.intersection_update(h_possibilities[h_index]) # using intersection_update which updates the set with the intersection of itself and another 
    if v_index is not None and v_index < len(v_possibilities):
        combined_possibilities.intersection_update(v_possibilities[v_index]) 

    # Return the combined possibilities for the current sequence
    return combined_possibilities 

# this function returns the possible values for a cell by combining the constraints from the horizontal and vertical sequences
def get_possible_values(board, row, col, h_constraints, v_constraints):
    possible_values = set(range(1, 10)) # intital possibilities
    # Propagate constraints from the current board state
    combined_possibilities = propagate_constraints(board, row, col, h_constraints, v_constraints)
    # The combined possibilities are for the current cell only
    possible_values.intersection_update(combined_possibilities)
    return possible_values # this is different from the one in the previous function because this one is for a single

# Testing get_possible_values
puzzle = {
    'board': [['X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X'],
              ['X', '.', '.', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X'],
              ['X', '.', '.', '.', 'X', '.', '.', 'X', 'X', 'X', 'X', '.', '.'],
              ['X', 'X', 'X', '.', '.', '.', '.', 'X', 'X', '.', '.', '.', '.'],
              ['X', 'X', '.', '.', '.', '.', 'X', 'X', '.', '.', '.', 'X', 'X'],
              ['X', 'X', '.', '.', 'X', '.', '.', 'X', '.', '.', 'X', 'X', 'X'],
              ['X', 'X', 'X', '.', '.', 'X', '.', '.', 'X', '.', '.', 'X', 'X'],
              ['X', 'X', 'X', '.', '.', 'X', '.', '.', 'X', '.', '.', 'X', 'X'],
              ['X', 'X', 'X', 'X', '.', '.', 'X', '.', '.', 'X', '.', '.', 'X'],
              ['X', 'X', 'X', '.', '.', '.', 'X', 'X', '.', '.', '.', '.', 'X'],
              ['X', '.', '.', '.', '.', 'X', 'X', '.', '.', '.', '.', 'X', 'X'],
              ['X', '.', '.', 'X', 'X', 'X', 'X', '.', '.', 'X', '.', '.', '.'],
              ['X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', '.', '.']],
    'h_constraints': [[11], [12, 8, 12], [22, 17], [19, 18], [17, 7, 16], [12, 11, 9],
                      [15, 7, 11], [13, 9, 5], [15, 23], [15, 14], [3, 3, 14], [9]],
    'v_constraints': [[3, 3], [17, 16, 4], [38, 3], [3, 35], [11, 16], [16, 7],
                      [23, 3], [16, 10], [35, 17], [4, 22], [4, 4, 3], [17, 17]],
    'id': 0
}

# print out the precieved horizonatal and vertical sum constraints for the cell (2,1) using the functions
h_word, h_target_sum = find_sequence_and_sum(puzzle['board'], 2, 1, puzzle['h_constraints'], True)
v_word, v_target_sum = find_sequence_and_sum(puzzle['board'], 2, 1, puzzle['v_constraints'], False)
print("Horizontal word:", h_word)
print("Horizontal target sum:", h_target_sum) # this should print out 12

print("Vertical word:", v_word)
print("Vertical target sum:", v_target_sum) # this should print out 3 
#print the possible values for the cell (2,1)
print("Possible values:", get_possible_values(puzzle['board'], 2, 1, puzzle['h_constraints'], puzzle['v_constraints'])) # this should print out {1, 2}


'''''
# print out the precieved horizonatal and vertical sum constraints for the cell (9,7) using the functions
h_word, h_target_sum = find_sequence_and_sum(puzzle['board'], 7, 9, puzzle['h_constraints'], True)
v_word, v_target_sum = find_sequence_and_sum(puzzle['board'], 9, 7, puzzle['v_constraints'], False)
print("Horizontal word:", h_word)
print("Horizontal target sum:", h_target_sum) # this should print out 14

print("Vertical word:", v_word)
print("Vertical target sum:", v_target_sum) # this should print out 3 
print("Possible values:", get_possible_values(puzzle['board'], 7, 9, puzzle['h_constraints'], puzzle['v_constraints'])) # this should print out {5, 6, 7, 8, 9}

#print the value of (9,7) in the board
print("Value in board:", puzzle['board'][7][9]) #
'''''


Horizontal word: ['.', '.', '.']
Horizontal target sum: 12
Vertical word: ['.', '.']
Vertical target sum: 3
Possible values: {1, 2}


'\'\'\n# print out the precieved horizonatal and vertical sum constraints for the cell (9,7) using the functions\nh_word, h_target_sum = find_sequence_and_sum(puzzle[\'board\'], 7, 9, puzzle[\'h_constraints\'], True)\nv_word, v_target_sum = find_sequence_and_sum(puzzle[\'board\'], 9, 7, puzzle[\'v_constraints\'], False)\nprint("Horizontal word:", h_word)\nprint("Horizontal target sum:", h_target_sum) # this should print out 14\n\nprint("Vertical word:", v_word)\nprint("Vertical target sum:", v_target_sum) # this should print out 3 \nprint("Possible values:", get_possible_values(puzzle[\'board\'], 7, 9, puzzle[\'h_constraints\'], puzzle[\'v_constraints\'])) # this should print out {5, 6, 7, 8, 9}\n\n#print the value of (9,7) in the board\nprint("Value in board:", puzzle[\'board\'][7][9]) #\n'

The possible values that cell (2,1 ) can take on should be  [1,2] because it has a horizontal constraint of 12 and a vertical constraint of 3

The possible values that cell (10,1 ) can take on should be  [1,2} because it has a horizontal constraint of 15 and a vertical constraint of 3

The possible values that cell (10, 2 ) can take on should be   because it has a horizontal constraint of 15 and a vertical constraint of 4

The possible values that cell (4, 2 ) can take on should be   because it has a horizontal constraint of 19 and a vertical constraint of 16

The possible values that cell (9, 7 ) can take on should be   because it has a horizontal constraint of 14 and a vertical constraint of 3